In [1]:
import numpy as np
import pandas as pd

In [2]:
dir ='/root/JupyterNotebook/kaggle-Redefining-Cancer-Treatment'

In [3]:
train_genevar = pd.read_csv(dir+'/4 input for stage2/training_variants.csv')
target = train_genevar['Class']

In [4]:
train_len= len(target)
print(train_len)

3689


In [5]:
test_genevar = pd.read_csv(dir+'/4 input for stage2/test_variants.csv')

In [6]:
print(len(test_genevar))

619


In [7]:
pid = test_genevar['ID'].values

In [8]:
import pickle
df_done = pickle.load(open(dir+'/5 middle for stage2/'+'onehotvar_cleantxt_data', 'rb'))

In [9]:
word_freq_allclass = pickle.load( open(dir+'/5 middle for stage2/'+'word_freq_all_data', 'rb'))

In [10]:
gen_var_lst = pickle.load(open(dir+'/5 middle for stage2/'+'gen_var_lst', 'rb'))

In [11]:
word_list = list(word_freq_allclass)

In [12]:
mylist = word_list+gen_var_lst

In [13]:
len(mylist)

9698

In [14]:
mylist = set(mylist)

In [15]:
my_vocabulary = {}
i = 0
for word in mylist:
    my_vocabulary[word] = i
    i +=1

In [16]:
print(len(my_vocabulary))

9250


In [17]:
print(len(df_done))

4308


In [19]:
traindata = df_done.iloc[:train_len]
testdata = df_done.iloc[train_len:]

In [20]:
from sklearn import preprocessing, pipeline, feature_extraction, decomposition, model_selection, metrics, cross_validation, svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import normalize, Imputer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

import sklearn
import pandas as pd
import numpy as np
import xgboost as xgb

import datetime
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV,train_test_split
from time import time
from sklearn import metrics
import sklearn

/root/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
class cust_regression_vals(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['CleanText'],axis=1).values
        return x

class cust_txt_col(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)

In [22]:
print('Pipeline...')
fp = pipeline.Pipeline([
    ('union', pipeline.FeatureUnion(
        n_jobs = -1,
        transformer_list = [
            ('pi3', pipeline.Pipeline([('CleanText', cust_txt_col('CleanText')), 
                                        ('hv', TfidfVectorizer(vocabulary=my_vocabulary)),
                                    ])),
            ('standard', cust_regression_vals())
           
        ])
    )])

Pipeline...


In [23]:
train = fp.fit_transform(traindata)
print (train.shape)

(3689, 9319)


In [24]:
test = fp.transform(testdata).tocsc()
print(test.shape)

(619, 9319)


In [25]:
y = target -1

In [27]:
import xgboost as xgb

denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'eta': 0.02,
        'max_depth': 6,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True,
        #new add
        'max_delta_step':5,
        'scale_pos_weight':10,
    }
    x1, x2, y1, y2 = train_test_split(train, y, test_size=0.15, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    score1 = metrics.log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
    print(score1)
    if denom != 0:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
        preds = pred.copy()
    denom += 1
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])


[0]	train-mlogloss:2.1538	valid-mlogloss:2.16213
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:1.07401	valid-mlogloss:1.36081
[100]	train-mlogloss:0.673685	valid-mlogloss:1.09757
[150]	train-mlogloss:0.47878	valid-mlogloss:0.988105
[200]	train-mlogloss:0.373627	valid-mlogloss:0.938538
[250]	train-mlogloss:0.30574	valid-mlogloss:0.916845
[300]	train-mlogloss:0.257987	valid-mlogloss:0.909957
[350]	train-mlogloss:0.222355	valid-mlogloss:0.908309
[400]	train-mlogloss:0.195539	valid-mlogloss:0.911889
[450]	train-mlogloss:0.172851	valid-mlogloss:0.917987
Stopping. Best iteration:
[352]	train-mlogloss:0.221212	valid-mlogloss:0.908172

0.90817159865
[0]	train-mlogloss:2.15382	valid-mlogloss:2.16185
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mloglos

In [28]:
submission['ID'] = pid
submission.to_csv(dir+'/result/stage2/double_valca_e0.02_depth6_delta5.csv', index=False)

In [29]:
import lightgbm as lgb
import matplotlib.pyplot as plt
lgb_params = {
    'learning_rate': 0.01,
    'max_depth': 5,
    'num_leaves': 40, 
    'objective': 'multiclass',
    'num_class':9,
    'tree_learner':'voting',
    'metric':'multi_logloss',
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'max_bin': 100}
# form LightGBM datasets
dtrain_lgb = lgb.Dataset(train, label=y)
# LightGBM, cross-validation
cv_result_lgb = lgb.cv(lgb_params, 
                       dtrain_lgb, 
                       num_boost_round=1000, 
                       nfold=5, 
                       stratified=True, 
                       early_stopping_rounds=50, 
                       verbose_eval=100, 
                       show_stdv=True)
num_boost_rounds_lgb = len(cv_result_lgb['multi_logloss-mean'])
print('num_boost_rounds_lgb=' + str(num_boost_rounds_lgb))
# train model
model_lgb = lgb.train(lgb_params, dtrain_lgb, num_boost_round=num_boost_rounds_lgb)
y_pred=model_lgb.predict(test)
classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
subm = pd.DataFrame(y_pred, columns=classes)
subm['ID'] = pid

subm.to_csv(dir+'/result/stage2/double_valca_lightgbm.csv', index=False)

[100]	cv_agg's multi_logloss: 1.3932 + 0.00881001
[200]	cv_agg's multi_logloss: 1.12378 + 0.0108234
[300]	cv_agg's multi_logloss: 0.998735 + 0.0111641
[400]	cv_agg's multi_logloss: 0.934519 + 0.0120051
[500]	cv_agg's multi_logloss: 0.900171 + 0.0127408
[600]	cv_agg's multi_logloss: 0.881366 + 0.0123839
[700]	cv_agg's multi_logloss: 0.87146 + 0.0118657
[800]	cv_agg's multi_logloss: 0.866593 + 0.0106844
[900]	cv_agg's multi_logloss: 0.864778 + 0.00919801
num_boost_rounds_lgb=940


In [26]:
t0 = time()
svd = decomposition.TruncatedSVD(n_components=300, n_iter=25, random_state=12)
train_af = svd.fit_transform(train)
print("done in %0.3fs"%(time() - t0)) 

done in 68.891s


In [27]:
t0 = time()  
test_af = svd.transform(test)  
print("done in %0.3fs"%(time() - t0))

done in 0.270s


In [28]:
train_af = preprocessing.scale(train_af)
test_af = preprocessing.scale(test_af)

In [29]:
x1, x2, y1, y2 = train_test_split(train_af, y, test_size = 0.15, random_state=11)

In [ ]:
from sklearn import svm

clf=svm.SVC(kernel='linear',probability=True,class_weight='balanced')

clf.fit(x1,y1)

print(clf.score(x2,y2))
pred = clf.predict_proba(test_af)

submission_svm = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])

In [31]:
submission_svm['ID'] = pid
submission_svm.to_csv(dir+'/result/stage2/double_valca_svm.csv', index=False)